# KoLIMA(MathAI) and BiLIMA

KoLIMA(MathAI) is a Korean translation of the [LIMA: Less Is More for Alignment](https://arxiv.org/pdf/2305.11206.pdf), created using Google's Gemini Pro 1.5.

While the [taeshahn/ko-lima](https://huggingface.co/datasets/taeshahn/ko-lima) dataset already exists, our KoLIMA(MathAI) dataset differs significantly in its use of Gemini Pro 1.5 for translation instead of the [DeepL API](https://developers.deepl.com/docs).
Furthermore, our dataset features user queries written in informal Korean (banmal, 반말) and assistant responses in formal Korean (jondaetmal, 존댓말).

BiLIMA is a bilingual LIMA dataset with two modes: `en_ko` and `ko_en`.
- `en_ko`: the user's query is given in English and the assistant's answer is given in Korean.
- `ko_en`: the user's query is given in Korean and the assistant's answer is given in English.

In [ ]:
from time import sleep
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset

import google.generativeai as genai

## LIMA

https://huggingface.co/datasets/GAIR/lima

### License

If the source data of LIMA has a stricter license than CC BY-NC-SA, the LIMA dataset follows the same. Otherwise, it follows the CC BY-NC-SA license.

In [ ]:
dataset = load_dataset("GAIR/lima")

## Generate KoLIMA(MathAI)

### Gemini Pro 1.5

You'll need a [Google Gemini API Key](https://aistudio.google.com/app/apikey) to run the following script.


In [ ]:
GOOGLE_API_KEY = ''

genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

system_instruction = "Translate the following English conversation into modern and natural Korean, following four rules:\n* For terminology, you can use the English word.\n* If there is no proper Korean word, then you can use the English word.\n* Each conversation turn is separated by [sep].\n* Translate the user's query (odd-numbered turns) into polite and friendly informal Korean (반말), and translate the assistant's responses (even-numbered turns) into formal Korean (존댓말).\n"

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

### Prepair Dataset

Each conversation turn is separated by `[sep]`.

In [ ]:
train_dataset_df = pd.DataFrame([{
  'conversations': str('\n[sep]\n'.join(data['conversations'])),
  'source': str(data['source']),
} for data in dataset['train']])

In [ ]:
print(train_dataset_df.loc[0,'conversations'])

### Generate

In [ ]:
def gen_ko_lima(
    lima_df:pd.DataFrame,
    file_path:str='./data/ko_lima.csv',
    resume:bool=False,
    first_sleep=10,
    second_sleep=20,
  ) -> None:
  if resume:
    train_dataset_ko_df = pd.read_csv(file_path)
  else:
    train_dataset_ko_df = lima_df.copy()
    train_dataset_ko_df.loc[:, 'korean_conversations'] = None
    
  idx = (train_dataset_ko_df.loc[:, 'korean_conversations'].isna())

  for i, (text, _, _) in tqdm(train_dataset_df[idx].iterrows(), total=len(train_dataset_df[idx])):
    print(f"({i})\n{text}")
    try:
      convo = model.start_chat(history=[])
      convo.send_message(text)
      ko_text = convo.last.text
    except:
      try:
        print(f"1st fail. ({i})")
        sleep(second_sleep)
        convo = model.start_chat(history=[])
        convo.send_message(text)
        ko_text = convo.last.text
      except:
        print(f"2nd fail. Pass ({i})")
        ko_text = ''
    if len(text.split("<sep>")) == len(ko_text.split("<sep>")):
      train_dataset_ko_df.loc[i, 'korean_conversations'] = ko_text
      train_dataset_ko_df.to_csv(file_path, index=False)
      print(f"({i})\n{ko_text}")
    else:
      print(f"({i}) ### Something's wrong!!! ###\n{ko_text}")
    sleep(first_sleep)

In [ ]:
gen_ko_lima(train_dataset_df)

### Resume Generation

In [ ]:
gen_ko_lima(train_dataset_df, resume=True)

## BiLIMA

In [ ]:
import pandas as pd

ko_lima_df = pd.read_csv('./data/ko_lima.csv')

In [ ]:
def _merge(en:str, ko:str):
  en_lst = [c.strip() for c in en.split('[sep]')]
  ko_lst = [c.strip() for c in ko.split('[sep]')]

  assert len(en_lst) == len(ko_lst)
  n = len(en_lst)
  en_ko = '[sep]'.join([ko_lst[i] if i%2 else en_lst[i] for i in range(n)])
  ko_en = '[sep]'.join([en_lst[i] if i%2 else ko_lst[i] for i in range(n)])

  return en_ko, ko_en

ko_lima_df.loc[:, ['en_ko', 'ko_en']] = ko_lima_df.loc[:, ['conversations', 'korean_conversations']].apply(_merge, axis=1)

ko_lima_df

In [ ]:
ko_lima_df.to_csv('./data/bi_lima.csv', index=False)